In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

matrix1 = tf.constant([[3,3]])
matrix2 = tf.constant([[2], [2]])

product = tf.matmul(matrix1, matrix2)

# sess = tf.Session()
# result = sess.run(product)
# print(result)
# sess.close()

with tf.Session() as sess:
    result2 = sess.run(product)
    print(result2)

[[12]]


In [2]:
state = tf.Variable(0, name='counter')
#print(state.name)

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for _ in range(5):
        sess.run(update)
        print(sess.run(state))

1
2
3
4
5


In [3]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1:[7.], input2:[2.]}))

[ 14.]


In [13]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' %n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weight = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name+'/weight', Weight)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name+'/biases', biases)
        with tf.name_scope('inputs'):
            Wx_plus_b = tf.matmul(inputs, Weight) + biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
            tf.summary.histogram(layer_name+'/outputs', outputs)
        return outputs
        
    

In [20]:
#Generating example data
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

# plt.scatter(x_data, y_data)
# plt.show()

# Define input placeholder for network
with tf.name_scope('input'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x-inputs')
    ys = tf.placeholder(tf.float32, [None, 1], name='y-inputs')

#Hidden layer
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)

#output layer
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)

#Error = real Data - prediction
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction), reduction_indices=[1]))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#Main part
init = tf.global_variables_initializer()
sess = tf.Session()
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('logs/',sess.graph)
sess.run(init)

for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        result = sess.run(merged,
                          feed_dict={xs: x_data, ys: y_data})
writer.add_summary(result, i)

#plot the data
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(x_data, y_data)
plt.ion()
plt.show()

# for i in range(1000):
#     #Training
#     sess.run(train_step, feed_dict={xs:x_data, ys:y_data})
#     if i % 50 == 0:
# #         result = sess.run(merged, feed_dict={xs:x_data, ys:y_data})
# #         writer.add_summary(result, i)
#         print(i, sess.run(loss, feed_dict={xs:x_data, ys:y_data}))
        
#         try:
#             ax.lines.remove(lines[0])
#         except Exception:
#             pass
            
#         prediction_value = sess.run(prediction, feed_dict={xs:x_data})
        
#         #plot the prediction
#         lines = ax.plot(x_data, prediction_value, 'r', lw=5)
#         plt.pause(0.9)

InvalidArgumentError: You must feed a value for placeholder tensor 'input_3/x-inputs' with dtype float
	 [[Node: input_3/x-inputs = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'input_3/x-inputs', defined at:
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/kernelapp.py", li